<a href="https://colab.research.google.com/github/BetanyaWoldgiorgis/Retention-Rates/blob/main/Copy_of_Sprint_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/ef2018b.csv")


In [7]:
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)

print(df.columns.tolist())


['unitid', 'efbage', 'line', 'lstudy', 'xefage01', 'efage01', 'xefage02', 'efage02', 'xefage03', 'efage03', 'xefage04', 'efage04', 'xefage05', 'efage05', 'xefage06', 'efage06', 'xefage07', 'efage07', 'xefage08', 'efage08', 'xefage09', 'efage09']


In [8]:
keep_cols = [
    "unitid",
    "lstudy",
    "efage01","efage02","efage03","efage04",
    "efage05","efage06","efage07","efage08","efage09"
]

print(df[keep_cols].head())

   unitid  lstudy  efage01  efage02  efage03  efage04  efage05  efage06  \
0  100654       1     2140     3352      241      373     5492      614   
1  100654       2     1967     2875      139      162     4842      301   
2  100654       5      173      477      102      211      650      313   
3  100654       1     1924     2974      116      132     4898      248   
4  100654       2     1869     2741      103       99     4610      202   

   efage07  efage08  efage09  
0     2381     3725     6106  
1     2106     3037     5143  
2      275      688      963  
3     2040     3106     5146  
4     1972     2840     4812  


In [9]:
df["retained"] = np.where(df["lstudy"] == 1, 1, 0)


In [10]:
df["pandemic_period"] = 0   # placeholder
df["post_pandemic"] = 0     # placeholder


In [11]:
X = df.drop(["lstudy", "retained"], axis=1)
y = df["retained"]


In [12]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)


In [13]:
from sklearn.dummy import DummyClassifier

baseline = DummyClassifier(strategy="most_frequent")
baseline.fit(X_train, y_train)

base_acc = baseline.score(X_test, y_test)
print("Baseline Accuracy:", base_acc)


Baseline Accuracy: 0.57156902908644


In [15]:
print(X.dtypes)


unitid              int64
efbage              int64
line                int64
xefage01           object
efage01             int64
xefage02           object
efage02             int64
xefage03           object
efage03             int64
xefage04           object
efage04             int64
xefage05           object
efage05             int64
xefage06           object
efage06             int64
xefage07           object
efage07             int64
xefage08           object
efage08             int64
xefage09           object
efage09             int64
pandemic_period     int64
post_pandemic       int64
dtype: object


In [16]:
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors="coerce")


In [17]:
X = X.fillna(0)


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y,
    test_size=0.30,
    stratify=y,
    random_state=42
)



In [20]:
lr = LogisticRegression(
    max_iter=1000,
    class_weight="balanced",
    solver="liblinear"
)

lr.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000, solver='liblinear')

In [21]:
# Make all features numeric
for col in X.columns:
    X[col] = pd.to_numeric(X[col], errors="coerce")

# Replace non-numeric flags like 'R' with 0
X = X.fillna(0)

# Resplit
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

# Refit model
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear")
lr.fit(X_train, y_train)


LogisticRegression(class_weight='balanced', max_iter=1000, solver='liblinear')

In [22]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=200,
    max_depth=20,
    min_samples_leaf=5,
    bootstrap=True,
    random_state=42
)
rf.fit(X_train, y_train)


RandomForestClassifier(max_depth=20, min_samples_leaf=5, n_estimators=200,
                       random_state=42)

XGBoost Model

In [23]:
pip install xgboost


In [24]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    learning_rate=0.1,
    max_depth=3,
    n_estimators=200,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42
)

xgb.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=10,
        verbose=False)


TypeError: XGBClassifier.fit() got an unexpected keyword argument 'early_stopping_rounds'

In [25]:
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report

def evaluate(model, X_test, y_test, name):
    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)[:,1]

    print(f"\n{name} Results")
    print("Accuracy:", accuracy_score(y_test, preds))
    print("ROC-AUC:", roc_auc_score(y_test, probs))
    print(confusion matrix:\n", confusion_matrix(y_test, preds))
    print(classification_report(y_test, preds))

evaluate(lr, X_test, y_test, "Logistic Regression")
evaluate(rf, X_test, y_test, "Random Forest")
evaluate(xgb, X_test, y_test, "XGBoost")


SyntaxError: invalid syntax. Perhaps you forgot a comma? (ipython-input-1311678584.py, line 10)